# Deep Learning - Assigment 1

Sagiv Melamed - I.D.
Dan Peled - I.D. 211547013

In [9]:
# imports
import numpy as np
# from util import *
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from typing import Callable


# Part 1
This section contains the implementations of the forward propagation

In [10]:
def initialize_parameters(layers_dims: list) -> dict:
    """
    Create an ANN architecture depending on layers_dims
    :param layers_dims: list of layers dimentions
    :type layers_dims: list
    :return: dictionary built as follows:
        W: list of matrices representing layer's weights, initialized randomly,
        b: list of biases for each layer, initialized to zero
    :rtype: dict
    """
    # Create W

    W_sizes = [(next_dim, current_dim) for current_dim, next_dim in zip(layers_dims[:-1], layers_dims[1:])]
    W = [np.random.randn(*Wi_size) for Wi_size in W_sizes]

    # create b

    b_sizes = layers_dims[1:]
    b = [np.zeros((1, bi_size)) for bi_size in b_sizes]

    return {
        "W": W,
        "b": b
    }


In [11]:
def linear_forward(A: np.ndarray, W: np.ndarray, B: np.ndarray) -> dict:
    """
    Performing linear forward on NN
    :param A: Activation vector of previous layer
    :type A: np.ndarray
    :param W: Weight matrix of the current layer
    :type W: np.ndarray
    :param B: Bias vector of the current layer
    :type B: np.ndarray
    :return: dictionary built as follows:
        Z: linear component of activation function
        linear_cache: A,W,B
    :rtype: dict
    """
    return {
        "Z": A.dot(W.T) + B,
        "linear_cache": {
            "A": A,
            "W": W,
            "B": B
        }
    }

Activation functions:

In [12]:
def softmax(Z: np.ndarray) -> dict:
    """
    Applying softmax on Z
    :param Z: the linear component of the activation function
    :type Z: np.ndarray
    :return: dictionary built as follows:
        A: Activation of th layer
        activation_cache: Z
    :rtype: dict
    """
    return {
        "A": np.exp(Z) / np.exp(Z).sum(),
        "activation_cahce": {
            "Z": Z
        }
    }


def relu(Z: np.ndarray) -> dict:
    """
        Applying relu on Z
        :param Z: the linear component of the activation function
        :type Z: np.ndarray
        :return: dictionary built as follows:
            A: Activation of th layer
            activation_cache: Z
        :rtype: dict
        """
    return {
        "A": np.maximum(0, Z),
        "activation_cahce": {
            "Z": Z
        }
    }


In [13]:
def linear_activation_forward(A_prev: np.ndarray, W: np.ndarray, B: np.ndarray,
                              activation: Callable[[np.ndarray], dict]) -> dict:
    cache = {}
    linear = linear_forward(A_prev, W, B)
    z, linear_cache = linear['Z'], linear['linear_cache']

    active = activation(z)
    a, activation_cache = active['A'], active['activation_cache']

    cache.update(linear_cache)
    cache.update(activation_cache)

    return {
        "A": a,
        "cache": cache
    }


In [14]:
def L_model_forward(X: np.ndarray, parameters: dict, use_batchnorm: bool = False):
    """

    :param X: matrix of inputs
    :type X: np.ndarray
    :param parameters: a dict like object containing W and b
    :type parameters: dict
    :param use_batchnorm: whether to use batch normalization or not
    :type use_batchnorm: bool
    :return:
        dictionary containing the activation of the ANN represented by the parameters on X and cache actions
    :rtype:
        dict
    """
    cache_list = list()
    A = X

    # Relu layers
    for W_i, b_i in zip(parameters["W"][:-1], parameters["b"][:-1]):
        results = linear_activation_forward(A, W_i, b_i, relu)
        A = results['A']
        if use_batchnorm:
            raise NotImplementedError()

        cache_list.append(results['cache'])

    # Softmax layer
    cache = {}
    Z, cache["linear_cache"] = list(linear_forward(A, parameters["W"][-1], parameters["b"][-1]).values())
    y, cache["activation_cache"] = list(softmax(Z).values())
    cache_list.append(cache)
    return y, cache_list

